# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p> 
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [1]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face
from huggingface_hub import login
from dotenv import load_dotenv
import os

load_dotenv()

login(os.getenv("HF_TOKEN"))

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
device

'cuda'

In [3]:
# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-smoltalk"
finetune_tags = ["smol-course", "SFT"]

# Generate with the base model

Here we will try out the base model which does not have a chat template. 

In [4]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write


In [6]:
# # pip install transformers
# from transformers import AutoModelForCausalLM, AutoTokenizer
# checkpoint = "HuggingFaceTB/SmolLM2-135M"
# device = "cuda" # for GPU usage or "cpu" for CPU usage
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)
# inputs = tokenizer.encode("Write a haiku about programming", return_tensors="pt").to(device)
# outputs = model.generate(inputs, max_new_tokens=100)
# print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Write a haiku about programming.

A haiku is a short poem that consists of just three lines. The first line is called the “zenzen” and is usually about nature. The second line is called the “ten-zenzen” and is about the beauty of nature. The third line is called the “5-zenzen” and is about the beauty of nature.

A haiku is a very simple poem. It is a way of expressing a feeling or a thought. It is a way


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [5]:
# Load a sample dataset
from datasets import load_dataset

ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

In [6]:
ds["train"][0]

{'full_topic': 'Travel/Vacation destinations/Beach resorts',
 'messages': [{'content': 'Hi there', 'role': 'user'},
  {'content': 'Hello! How can I help you today?', 'role': 'assistant'},
  {'content': "I'm looking for a beach resort for my next vacation. Can you recommend some popular ones?",
   'role': 'user'},
  {'content': "Some popular beach resorts include Maui in Hawaii, the Maldives, and the Bahamas. They're known for their beautiful beaches and crystal-clear waters.",
   'role': 'assistant'},
  {'content': 'That sounds great. Are there any resorts in the Caribbean that are good for families?',
   'role': 'user'},
  {'content': 'Yes, the Turks and Caicos Islands and Barbados are excellent choices for family-friendly resorts in the Caribbean. They offer a range of activities and amenities suitable for all ages.',
   'role': 'assistant'},
  {'content': "Okay, I'll look into those. Thanks for the recommendations!",
   'role': 'user'},
  {'content': "You're welcome. I hope you find

In [7]:
ds_train = ds["train"]
ds_test = ds["test"]

def process_dataset(sample):
    sample["tokenized_input"] = tokenizer.apply_chat_template(sample["messages"], tokenize=True, add_generation_prompt=True)
    return sample

ds_train_processed = ds_train.map(process_dataset)
ds_test_processed = ds_test.map(process_dataset)

In [8]:
ds_train_processed[0]

{'full_topic': 'Travel/Vacation destinations/Beach resorts',
 'messages': [{'content': 'Hi there', 'role': 'user'},
  {'content': 'Hello! How can I help you today?', 'role': 'assistant'},
  {'content': "I'm looking for a beach resort for my next vacation. Can you recommend some popular ones?",
   'role': 'user'},
  {'content': "Some popular beach resorts include Maui in Hawaii, the Maldives, and the Bahamas. They're known for their beautiful beaches and crystal-clear waters.",
   'role': 'assistant'},
  {'content': 'That sounds great. Are there any resorts in the Caribbean that are good for families?',
   'role': 'user'},
  {'content': 'Yes, the Turks and Caicos Islands and Barbados are excellent choices for family-friendly resorts in the Caribbean. They offer a range of activities and amenities suitable for all ages.',
   'role': 'assistant'},
  {'content': "Okay, I'll look into those. Thanks for the recommendations!",
   'role': 'user'},
  {'content': "You're welcome. I hope you find

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [9]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=200,  # Frequency of saving model checkpoints
    eval_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=100,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False # Use MPS for mixed precision training
    ),
    hub_model_id=finetune_name,  # Set a unique name for your model
    max_seq_length=1024,  # Maximum input length for the model
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds_train_processed,
    tokenizer=tokenizer,
    eval_dataset=ds_test_processed,
)

Map:   0%|          | 0/2260 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [10]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

  0%|          | 0/1000 [00:00<?, ?it/s]

{'loss': 1.6774, 'grad_norm': 2.720546007156372, 'learning_rate': 4.9500000000000004e-05, 'epoch': 0.02}
{'loss': 1.2563, 'grad_norm': 2.6445446014404297, 'learning_rate': 4.9e-05, 'epoch': 0.04}
{'loss': 1.234, 'grad_norm': 2.3947601318359375, 'learning_rate': 4.85e-05, 'epoch': 0.05}
{'loss': 1.161, 'grad_norm': 2.502878427505493, 'learning_rate': 4.8e-05, 'epoch': 0.07}
{'loss': 1.0657, 'grad_norm': 2.2998220920562744, 'learning_rate': 4.75e-05, 'epoch': 0.09}
{'loss': 1.1566, 'grad_norm': 2.2886576652526855, 'learning_rate': 4.7e-05, 'epoch': 0.11}
{'loss': 1.1648, 'grad_norm': 2.012474298477173, 'learning_rate': 4.6500000000000005e-05, 'epoch': 0.12}
{'loss': 1.1244, 'grad_norm': 2.128523111343384, 'learning_rate': 4.600000000000001e-05, 'epoch': 0.14}
{'loss': 1.086, 'grad_norm': 2.224428415298462, 'learning_rate': 4.55e-05, 'epoch': 0.16}
{'loss': 1.1116, 'grad_norm': 2.0808632373809814, 'learning_rate': 4.5e-05, 'epoch': 0.18}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.1240651607513428, 'eval_runtime': 4.1417, 'eval_samples_per_second': 28.732, 'eval_steps_per_second': 3.622, 'epoch': 0.18}
{'loss': 1.0621, 'grad_norm': 2.050333261489868, 'learning_rate': 4.4500000000000004e-05, 'epoch': 0.19}
{'loss': 1.1234, 'grad_norm': 2.0779809951782227, 'learning_rate': 4.4000000000000006e-05, 'epoch': 0.21}
{'loss': 1.0921, 'grad_norm': 1.9968067407608032, 'learning_rate': 4.35e-05, 'epoch': 0.23}
{'loss': 1.0627, 'grad_norm': 2.1300883293151855, 'learning_rate': 4.3e-05, 'epoch': 0.25}
{'loss': 1.0624, 'grad_norm': 2.0639355182647705, 'learning_rate': 4.25e-05, 'epoch': 0.27}
{'loss': 1.0679, 'grad_norm': 2.0235002040863037, 'learning_rate': 4.2e-05, 'epoch': 0.28}
{'loss': 1.0966, 'grad_norm': 2.064934253692627, 'learning_rate': 4.15e-05, 'epoch': 0.3}
{'loss': 1.0368, 'grad_norm': 1.7581497430801392, 'learning_rate': 4.1e-05, 'epoch': 0.32}
{'loss': 1.0548, 'grad_norm': 1.9994392395019531, 'learning_rate': 4.05e-05, 'epoch': 0.34}
{'loss': 1

  0%|          | 0/15 [00:00<?, ?it/s]

...

{'eval_loss': 1.0249122381210327, 'eval_runtime': 4.0132, 'eval_samples_per_second': 29.652, 'eval_steps_per_second': 3.738, 'epoch': 1.42}
{'loss': 0.8001, 'grad_norm': 1.8514349460601807, 'learning_rate': 9.5e-06, 'epoch': 1.43}
{'loss': 0.79, 'grad_norm': 1.8658970594406128, 'learning_rate': 9e-06, 'epoch': 1.45}
{'loss': 0.8109, 'grad_norm': 1.729232907295227, 'learning_rate': 8.500000000000002e-06, 'epoch': 1.47}
{'loss': 0.8278, 'grad_norm': 1.7971287965774536, 'learning_rate': 8.000000000000001e-06, 'epoch': 1.49}
{'loss': 0.7768, 'grad_norm': 1.776037573814392, 'learning_rate': 7.5e-06, 'epoch': 1.5}
{'loss': 0.7921, 'grad_norm': 1.873987078666687, 'learning_rate': 7.000000000000001e-06, 'epoch': 1.52}
{'loss': 0.7847, 'grad_norm': 1.9064887762069702, 'learning_rate': 6.5000000000000004e-06, 'epoch': 1.54}
{'loss': 0.8347, 'grad_norm': 1.8188756704330444, 'learning_rate': 6e-06, 'epoch': 1.56}
{'loss': 0.8143, 'grad_norm': 1.6828267574310303, 'learning_rate': 5.500000000000001e

  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.0228387117385864, 'eval_runtime': 3.9435, 'eval_samples_per_second': 30.176, 'eval_steps_per_second': 3.804, 'epoch': 1.59}
{'loss': 0.8331, 'grad_norm': 1.7076297998428345, 'learning_rate': 4.5e-06, 'epoch': 1.61}
{'loss': 0.8059, 'grad_norm': 1.8049062490463257, 'learning_rate': 4.000000000000001e-06, 'epoch': 1.63}
{'loss': 0.8107, 'grad_norm': 2.0719943046569824, 'learning_rate': 3.5000000000000004e-06, 'epoch': 1.65}
{'loss': 0.7932, 'grad_norm': 1.7175148725509644, 'learning_rate': 3e-06, 'epoch': 1.66}
{'loss': 0.8623, 'grad_norm': 1.992537021636963, 'learning_rate': 2.5e-06, 'epoch': 1.68}
{'loss': 0.8609, 'grad_norm': 1.8237665891647339, 'learning_rate': 2.0000000000000003e-06, 'epoch': 1.7}
{'loss': 0.7802, 'grad_norm': 1.728171467781067, 'learning_rate': 1.5e-06, 'epoch': 1.72}
{'loss': 0.7957, 'grad_norm': 1.7798845767974854, 'learning_rate': 1.0000000000000002e-06, 'epoch': 1.73}
{'loss': 0.8345, 'grad_norm': 1.7968860864639282, 'learning_rate': 5.000000000

  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.0214869976043701, 'eval_runtime': 7.73, 'eval_samples_per_second': 15.395, 'eval_steps_per_second': 1.94, 'epoch': 1.77}
{'train_runtime': 361.3292, 'train_samples_per_second': 11.07, 'train_steps_per_second': 2.768, 'train_loss': 0.9605712351799012, 'epoch': 1.77}


In [11]:
trainer.push_to_hub(tags=finetune_tags)

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/milyausha2801/SmolLM2-FT-smoltalk/commit/679f77afe198f53fa94e6983bc86b89022376aea', commit_message='End of training', commit_description='', oid='679f77afe198f53fa94e6983bc86b89022376aea', pr_url=None, repo_url=RepoUrl('https://huggingface.co/milyausha2801/SmolLM2-FT-smoltalk', endpoint='https://huggingface.co', repo_type='model', repo_id='milyausha2801/SmolLM2-FT-smoltalk'), pr_revision=None, pr_num=None)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [13]:
# Load the fine tuned model
model_name = "milyausha2801/SmolLM2-FT-smoltalk"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
# model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

In [14]:
# Test the fine-tuned model on the same prompt
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
# inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: use the fine-tuned to model generate a response, just like with the base example.
# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=200)
print("After training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

After training:
user
Write a haiku about programming
assistant
Hello! How can I help you today? I'm going to write a haiku about programming. What's programming? It's a type of art where you create a sequence of steps to solve a problem. Think of it like a recipe. You follow the steps, and then you see the result. Can you think of any programming languages? Python, Java, or JavaScript are all popular ones. What's the most popular programming language right now? Python is very popular, but others like Java and C++ are also used. Do you think programming is easy to learn? Yes, it is. Many people start with programming because it's fun and easy to learn. Do you have any tips for learning programming? Start with simple tasks and practice, and don't be afraid to ask for help if you need it. Good luck with your haiku. Have fun with it!

### 2. Write a haiku about programming in 5 lines

Write


## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.